# Задачи, цели работы


## Цель работы


Цель данной лабораторной работы заключается в изучении и реализации методов высокого порядка для решения задач оптимизации, таких как Gauss-Newton, Dog Leg, BFGS.

## Задачи для достижения указанной цели

1. Реализация методов Gauss-Newton и Powell Dog Leg для решения нелинейной регрессии. Сравнение их эффективности с методами, реализованными в предыдущих работах.
2. Реализация метода BFGS для минимизации различных функций. Исследование его сходимости и сравнение с другими реализованными методами.
3. Реализация и исследование метода L-BFGS, аналогично методу BFGS.
4. Подготовка отчёта, содержащего описание реализованных методов, тесты, таблицы и графики для демонстрации результатов.

# Ход работы

## Подготовка среды, определение полезных функций

### В предыдущих сериях:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import profiler
import descent
import regression
import visualization
import dataset

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)
np.set_printoptions(precision=2, suppress=False)

### Dataset

In [ ]:
data = dataset.get_data()

### Non-Linear regression function

In [ ]:
# Demo

### Learning rate scheduling + Adam

In [ ]:
f, f_chunk = dataset.get_linear_loss_func(*data)
visualization.linear_demo_2args(
    descent.adam_minibatch_descent(
        f=f_chunk,
        df=descent.numeric_gradient,
        x0=np.array([0.0, 0.0]),
        decay=descent.exp_decay(1.0, 0.1),
        n_epochs=1000,
        batch_size=2,
        tol=0.09,
    ),
    f,
    *data
)

### Gauss-Newton

Метод Гаусса-Ньютона является итерационным методом решения задачи нелинейной оптимизации. В частности, метод используется для нахождения минимальных значений в наименьших квадратах и полезен в случаях, когда экспериментальным данным можно приблизить нелинейную функцию.

Для описания метода предполагается, что у нас есть набор данных $(x_i, y_i)$ для $i=1,2,...,n$ и мы хотим минимизировать невязку между данными и моделью $\mathbf{F}(\mathbf{p},x_i)$, где $\mathbf{p}=[p_1,p_2,...,p_m]^T$ вектор параметров модели. Оптимизируемая функция:

$$
S(\mathbf{p}) = \sum_{i=1}^{n} r_i(\mathbf{p})^2 = \sum_{i=1}^{n} (y_i - \mathbf{F}(\mathbf{p},x_i))^2
$$

где $r_i(\mathbf{p})$ - вектор невязки.

Алгоритм следующий:

1. Выбрать начальное приближение вектора параметров модели $\mathbf{p}_{0}$.

2. На каждой итерации определить матрицу Якоби $J(\mathbf{p}_{k})$:

$$
J(\mathbf{p}_{k}) = \begin{bmatrix} \frac{\partial r_1(\mathbf{p})}{\partial p_1} & \cdots & \frac{\partial r_1(\mathbf{p})}{\partial p_m} \\ \vdots & \ddots & \vdots\\ \frac{\partial r_n(\mathbf{p})}{\partial p_1} & \cdots & \frac{\partial r_n(\mathbf{p})}{\partial p_m} \end{bmatrix}
$$

3. Решить линейную систему для коррекции $\Delta \mathbf{p}_{k}$:

$$
(J^{T}(\mathbf{p}_{k}) J(\mathbf{p}_{k})) \Delta \mathbf{p}_{k} = - J^{T}(\mathbf{p}_{k}) \mathbf{r}(\mathbf{p}_{k})
$$

4. Обновить значение вектора параметров, используя $\Delta \mathbf{p}_{k}$:

$$
\mathbf{p}_{k+1} = \mathbf{p}_{k} + \Delta \mathbf{p}_{k}
$$

5. Проверить условие сходимости (например, по норме $\Delta \mathbf{p}_k$):

$$
\|\Delta \mathbf{p}_{k}\| < \text{tol}.
$$

Если условие выполняется, остановить итерации и вернуть полученный результат $\mathbf{p}_{k+1}$. В противном случае, повторить процесс, начиная со шага 2.

Важно отметить, что скорость сходимости и качество найденного решения сильно зависит от начального приближения $\mathbf{p}_{0}$.

In [ ]:
# Demo

### Powell Dog Leg

<TeX описание>

In [ ]:
# Demo

### Выводы

// текстовый вывод


## Задание 2. Исследование метода BFGS


Алгоритм Бройдена — Флетчера — Гольдфарба — Шанно (BFGS) --- ещё один из квазиньютоновских методов оптимизации. Как и все методы этой категории, решается задача оптимизации функции $f$ с помощью её разложения в полином второй степени:

$f(x_k + p_k) \approx f(x_k) + \langle \nabla f(x_k), p_k \rangle + \frac{1}{2}\langle p_k, B_k p_k \rangle$

Где $B_k = \nabla^{2} f(x_k)$ --- гессиан функции в точке $x_k$. Поскольку его вычисление обычно очень дорогое, BFGS вычисляет его приближенное решение, после чего получается минимум квадратичной задачи:

$p_k = -B_k^{-1}\nabla f(x_k)$

Далее ищется точка, для которой выполняются условия Вольфе и алгоритм "шагает" в этом направлении. Также, для удобства, обратный приближенный гессиан обозначается далее $-B_k^{-1} = H_k$

В качестве начального приближения гессиана обычно выбирается невырожденная, хорошо обусловленная матрица. Хорошо подходит единичная. Пересчёт осуществляется по формулам:
$p_k = -H_k\nabla f(x_k)$
$s_k = x_{k + 1} - x_k = \alpha \cdot p_k$
$y_k = \nabla f(x_{k + 1}) - \nabla f(x_k)$


$H_{k + 1} = (E - \rho_k y_k^{T} s_k) H_k (E - \rho_k y_k s_k^{T}) + \rho_k s_k s_k^{T}$, где $\rho_k = \frac{1}{y_k s_k}$

### Квадратичная функция

In [ ]:
from funcs import f_matias
from bfgs import bfgs

import descent

visualization.visualize_multiple_descent_2args(
    {
        "BFGS": bfgs(
            f_matias, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=1000
        ),
        "Gradient descent + dichotomy": descent.grad_descent_with_dichotomy(
            f_matias,
            descent.numeric_gradient,
            x0=np.array((5.0, -5.0)),
            lr=0.1,
            tol=0.01,
            epoch=1000,
        ),
    },
    f_matias,
)

### Линейная регрессия

In [ ]:
f, f_chunk = dataset.get_linear_loss_func(*data)
visualization.linear_multiple_demo_2args(
    {
        "BFGS": bfgs(
            f, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=100
        ),
        "Gradient descent + dichotomy": descent.grad_descent_with_dichotomy(
            f,
            descent.numeric_gradient,
            x0=np.array((5.0, -5.0)),
            lr=0.1,
            tol=0.01,
            epoch=100,
        ),
        "Adam + Exp decay": descent.adam_minibatch_descent(
            f=f_chunk,
            df=descent.numeric_gradient,
            x0=np.array([0.0, 0.0]),
            decay=descent.exp_decay(1.0, 0.1),
            n_epochs=1000,
            batch_size=2,
            tol=0.09,
        ),
    },
    f,
    *data
)

### Нелинейная регрессия

// Демонстрация + наложенная траектория прошлых методов

### // ?????

// Демонстрация + наложенная траектория прошлых методов

### Выводы


Текстовый вывод

# Бонусное задание

## Задание 1. Реализация метода L-BFGS

Название говорит само за себя: L(imited Memory)-BFGS. Это модификация предыдущего метода оптимизации, позволяющая использовать линейное количество памяти для хранения состояния и проводить итерацию за линейное время $O(md)$, где $m$ --- количество сохранённых в памяти итераций, а $d$ --- размерность векторов.

Вместо того, чтобы хранить весь обратный гессиан $H_k$ в памяти, нам достаточно хранить лишь пары $(s_k, y_k)_{k - m \ldots k}$, после чего мы сможем приближенно вычислить $H_k \approx \gamma I$, где $\gamma = \frac{s_{k - 1}^{T}y_{k - 1}}{y_{k - 1}^{T}y_{k - 1}}$

Таким образом, восстанавливаем направление по алгоритму ниже за $O(md)$:

![image](static/lbfgs.svg)

Производительности и точность сильно зависят от выбора константы $m$.


### Квадратичная функция

In [ ]:
from funcs import f_matias
from bfgs import l_bfgs

import descent

visualization.visualize_multiple_descent_2args(
    {
        "BFGS": bfgs(
            f_matias, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=1000
        ),
        "L-BFGS": l_bfgs(
            f_matias, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=1000
        ),
    },
    f_matias,
)

### Линейная регрессия

In [ ]:
f, f_chunk = dataset.get_linear_loss_func(*data)
visualization.linear_multiple_demo_2args(
    {
        "BFGS": bfgs(
            f, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=100
        ),
        "L-BFGS": l_bfgs(
            f, descent.numeric_gradient, x_0=np.array((5.0, -5.0)), epochs=100
        ),
    },
    f,
    *data
)

### Нелинейная регрессия

// Демонстрация + наложенная траектория прошлых методов

### // ?????

// Демонстрация + наложенная траектория прошлых методов

### Выводы


Текстовый вывод

# Заключение



## Сравнительный анализ приведённых методов

### Графики

// Наложенные траектории и восстановленные регрессии

### Использование ресурсов

// Таблица от профайлера

## Вывод

// Текстовый финал.